In [1]:
# @title 1. Create Project Structure
import os

folders = [
    "wpf_engine",
    "wpf_engine/config",
    "wpf_engine/core",
    "wpf_engine/data",
    "wpf_engine/models",
    "wpf_engine/pipelines",
    "wpf_engine/utils",
    "wpf_engine/scripts",
    "saved_models"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    # Create __init__.py for module resolution
    if "saved_models" not in folder:
        with open(os.path.join(folder, "__init__.py"), "w") as f:
            pass

print("✅ Modular directory structure created successfully.")

✅ Modular directory structure created successfully.


In [2]:
# @title 2. Configuration & Logging (%%writefile wpf_engine/config/settings.py)
%%writefile wpf_engine/config/settings.py
import os

class Config:
    # Paths
    DATA_PATH = "wtbdata_245days.csv"
    SAVE_DIR = "saved_models"

    # Spatio-Temporal Specs
    LOOKBACK_STEPS = 144  # 24 hours of 10-min intervals
    FORECAST_STEPS = 288  # 48 hours of 10-min intervals
    TRAIN_STRIDE = 6      # Slide window by 1 hour during training to prevent RAM explosion

    # Neural Architecture Config
    BATCH_SIZE = 256
    EPOCHS = 30
    LEARNING_RATE = 0.001

    # Features
    TARGET_COL = "Patv"
    PHYSICS_FEATURES = ["U", "V", "Energy_Flux", "Momentum", "Etmp", "Itmp", "Ndir", "Pab1", "Pab2", "Pab3", "Prtv"]

    @classmethod
    def get_model_path(cls, version="v1"):
        return os.path.join(cls.SAVE_DIR, f"physics_ai_engine_{version}.h5")

    @classmethod
    def get_scaler_path(cls, version="v1"):
        return os.path.join(cls.SAVE_DIR, f"scaler_{version}.pkl")

Writing wpf_engine/config/settings.py


In [3]:
# @title 3. Robust Logger & Exceptions (%%writefile wpf_engine/utils/logger.py)
%%writefile wpf_engine/utils/logger.py
import logging
import sys

def get_logger(name="WPF_Engine"):
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

        ch = logging.StreamHandler(sys.stdout)
        ch.setFormatter(formatter)
        logger.addHandler(ch)

        fh = logging.FileHandler("engine_run.log")
        fh.setFormatter(formatter)
        logger.addHandler(fh)
    return logger

class DataIntegrityError(Exception):
    """Raised when SCADA data violates expected physics/rules boundaries."""
    pass

Writing wpf_engine/utils/logger.py


In [4]:
# @title 4. Adapter & Physics Engine (%%writefile wpf_engine/data/physics_engine.py)
%%writefile wpf_engine/data/physics_engine.py
import numpy as np
import pandas as pd
from wpf_engine.utils.logger import get_logger

logger = logging.getLogger("WPF_Engine") if 'logging' in locals() else get_logger()

class SDWPFAdapter:
    """Ingests raw SCADA logs, applies KDD Cup 2022 validity rules."""

    @staticmethod
    def clean_scada_data(df: pd.DataFrame) -> pd.DataFrame:
        logger.info("Applying KDD Cup 2022 Boundary & Cleaning Rules...")

        # Rule 1: Patv < 0 -> Patv = 0
        df['Patv'] = np.where(df['Patv'] < 0, 0, df['Patv'])

        # Rule 2: Invalid Pitch Angles -> Nullify Target (Turbine Stopped/Abnormal)
        invalid_pitch = (df['Pab1'] > 89) | (df['Pab2'] > 89) | (df['Pab3'] > 89)

        # Rule 3: Abnormal Nacelle and Wind Directions
        invalid_ndir = (df['Ndir'] > 720) | (df['Ndir'] < -720)
        invalid_wdir = (df['Wdir'] > 180) | (df['Wdir'] < -180)

        # Combine rules and mask abnormal targets
        abnormal_mask = invalid_pitch | invalid_ndir | invalid_wdir
        df.loc[abnormal_mask, 'Patv'] = np.nan

        # Forward fill missing values (up to a limit) to maintain sequence integrity
        df.ffill(limit=6, inplace=True) # Max 1 hour of imputation
        df.bfill(inplace=True)
        return df

class PhysicsEngine:
    """Transforms raw meteorological readings into continuous physics states."""

    @staticmethod
    def apply_physics(df: pd.DataFrame) -> pd.DataFrame:
        logger.info("Igniting Physics Engine: Calculating vectors, flux, and momentum...")

        # 1. Wind Vectors (U, V) - Resolves the 359/1 degree circular boundary AI confusion
        wind_dir_rad = np.deg2rad(df['Wdir'])
        df['U'] = df['Wspd'] * np.cos(wind_dir_rad)
        df['V'] = df['Wspd'] * np.sin(wind_dir_rad)

        # 2. Energy Flux (V^3) - Wind power is proportional to the cube of velocity
        df['Energy_Flux'] = np.power(df['Wspd'], 3)

        # 3. Momentum (Delta V) - Acceleration indicates incoming gusts/fronts
        # Group by TurbID to prevent cross-turbine momentum calculations
        df['Momentum'] = df.groupby('TurbID')['Wspd'].diff().fillna(0)

        return df

Writing wpf_engine/data/physics_engine.py


In [5]:
# @title 5. Vectorized Sequence Generator (%%writefile wpf_engine/data/sequence_generator.py)
%%writefile wpf_engine/data/sequence_generator.py
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
from wpf_engine.config.settings import Config
from wpf_engine.utils.logger import get_logger

logger = get_logger()

class VectorizedSlicer:
    """Generates 3D tensors natively in RAM, respecting turbine boundaries."""

    def __init__(self):
        self.scaler = StandardScaler()
        self.target_col = Config.TARGET_COL
        self.feature_cols = Config.PHYSICS_FEATURES

    def fit_transform_scale(self, df: pd.DataFrame, is_train: bool = True):
        logger.info("Scaling features...")
        if is_train:
            df[self.feature_cols] = self.scaler.fit_transform(df[self.feature_cols])
            joblib.dump(self.scaler, Config.get_scaler_path())
        else:
            self.scaler = joblib.load(Config.get_scaler_path())
            df[self.feature_cols] = self.scaler.transform(df[self.feature_cols])
        return df

    def create_sequences(self, df: pd.DataFrame):
        logger.info("Executing Vectorized RAM Slicing per Turbine...")
        X, Y, Anchors = [], [], []

        grouped = df.groupby('TurbID')

        for turb_id, group in grouped:
            group = group.reset_index(drop=True)
            values_x = group[self.feature_cols].values
            values_y = group[self.target_col].values

            # Using strided window logic
            num_samples = len(group) - Config.LOOKBACK_STEPS - Config.FORECAST_STEPS
            if num_samples <= 0:
                continue

            for i in range(0, num_samples, Config.TRAIN_STRIDE):
                # Historical Features
                x_seq = values_x[i : i + Config.LOOKBACK_STEPS]
                # Target Future Actual Power
                y_seq = values_y[i + Config.LOOKBACK_STEPS : i + Config.LOOKBACK_STEPS + Config.FORECAST_STEPS]
                # The "Power Anchor": Last known power value at the end of lookback
                anchor = values_y[i + Config.LOOKBACK_STEPS - 1]

                # We skip sequences where the target has NaNs (due to invalid rules)
                if np.isnan(y_seq).any() or np.isnan(anchor):
                    continue

                X.append(x_seq)
                Y.append(y_seq)
                Anchors.append([anchor]) # Shape (1,)

        logger.info(f"Generated {len(X)} sequences.")
        return np.array(X, dtype=np.float32), np.array(Anchors, dtype=np.float32), np.array(Y, dtype=np.float32)

Writing wpf_engine/data/sequence_generator.py


In [6]:
# @title 6. Core AI Models (%%writefile wpf_engine/models/hybrid_model.py)
%%writefile wpf_engine/models/hybrid_model.py
import tensorflow as tf
from tensorflow.keras import layers, Model
from wpf_engine.config.settings import Config
from wpf_engine.utils.logger import get_logger

logger = get_logger()

class DifferentialTransformer:
    @staticmethod
    def build_model(input_shape=(Config.LOOKBACK_STEPS, len(Config.PHYSICS_FEATURES))):
        logger.info("Constructing Residual Differential Transformer...")

        # Input 1: Historical Physics Features
        inputs_seq = layers.Input(shape=input_shape, name="historical_physics")

        # Input 2: Power Anchor (The exact power output at time T=0)
        input_anchor = layers.Input(shape=(1,), name="power_anchor")

        # Stage 1: Edge Detection (Gusts)
        x = layers.Conv1D(filters=64, kernel_size=3, padding="same", activation="relu")(inputs_seq)
        x = layers.MaxPooling1D(pool_size=2)(x)

        # Stage 2: Temporal Memory (Weather Fronts)
        x = layers.Bidirectional(layers.GRU(128, return_sequences=True))(x)

        # Stage 3: Attention Mechanism (Focusing on cyclical patterns)
        attention = layers.MultiHeadAttention(num_heads=4, key_dim=128)(x, x)
        x = layers.Add()([x, attention]) # Residual connection
        x = layers.LayerNormalization()(x)

        # Collapse sequence to vector
        x = layers.GlobalAveragePooling1D()(x)

        # Stage 4: Differential Head (Predicting Delta -> Change in Power)
        x = layers.Dense(256, activation="relu")(x)
        x = layers.Dropout(0.2)(x)
        delta_power = layers.Dense(Config.FORECAST_STEPS, activation="linear", name="delta_prediction")(x)

        # Stage 5: The "Game Changing" Anchor Addition
        # Expand anchor to match forecast horizon: shape (Batch, 288)
        repeated_anchor = layers.RepeatVector(Config.FORECAST_STEPS)(input_anchor)
        repeated_anchor = layers.Reshape((Config.FORECAST_STEPS,))(repeated_anchor)

        # Final Output = Present Power + Predicted Change
        final_prediction = layers.Add(name="final_absolute_power")([repeated_anchor, delta_power])

        # Enforce physical constraints (Power cannot realistically be < 0)
        final_prediction = layers.ReLU()(final_prediction)

        model = Model(inputs=[inputs_seq, input_anchor], outputs=final_prediction)

        # We use MSE for strict gradient descent, but log MAE for human readability
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=Config.LEARNING_RATE),
                      loss='mse',
                      metrics=['mae'])
        return model

Writing wpf_engine/models/hybrid_model.py


In [7]:
# @title 7. Training Pipeline (%%writefile wpf_engine/pipelines/trainer.py)
%%writefile wpf_engine/pipelines/trainer.py
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from wpf_engine.config.settings import Config
from wpf_engine.data.physics_engine import SDWPFAdapter, PhysicsEngine
from wpf_engine.data.sequence_generator import VectorizedSlicer
from wpf_engine.models.hybrid_model import DifferentialTransformer
from wpf_engine.utils.logger import get_logger

logger = get_logger()

class EnginePipeline:
    def __init__(self):
        self.slicer = VectorizedSlicer()

    def run_training(self):
        logger.info(f"Loading Raw SCADA Data from {Config.DATA_PATH}...")
        try:
            # We sample chunk if dataset is massive to prevent Colab crashing initially,
            # though 250MB should fit. Adjust nrows if needed.
            df = pd.read_csv(Config.DATA_PATH)
        except FileNotFoundError:
            logger.error(f"Dataset not found at {Config.DATA_PATH}. Please upload it to Colab.")
            return

        # 1. Pipeline Ingestion
        df = SDWPFAdapter.clean_scada_data(df)
        df = PhysicsEngine.apply_physics(df)
        df = df.dropna().reset_index(drop=True)

        # 2. Scaling
        df = self.slicer.fit_transform_scale(df, is_train=True)

        # 3. RAM Slicing
        X, Anchors, Y = self.slicer.create_sequences(df)

        if len(X) == 0:
            logger.error("No valid sequences generated. Check data integrity.")
            return

        # 4. Initialize Brain
        model = DifferentialTransformer.build_model()
        model.summary(print_fn=logger.info)

        # 5. Callbacks for Industrial Robustness
        callbacks = [
            EarlyStopping(patience=5, restore_best_weights=True, monitor='val_mae'),
            ModelCheckpoint(Config.get_model_path(), save_best_only=True, monitor='val_mae'),
            ReduceLROnPlateau(factor=0.5, patience=2, min_lr=1e-6)
        ]

        # 6. Execution
        logger.info("Initializing GPU Training Loop...")
        history = model.fit(
            x=[X, Anchors],
            y=Y,
            batch_size=Config.BATCH_SIZE,
            epochs=Config.EPOCHS,
            validation_split=0.15,
            callbacks=callbacks,
            shuffle=True
        )
        logger.info(f"Training Complete. Model securely locked at {Config.get_model_path()}")

Writing wpf_engine/pipelines/trainer.py


In [8]:
# @title 8. Execution Script (%%writefile run_engine.py)
%%writefile run_engine.py
from wpf_engine.pipelines.trainer import EnginePipeline
from wpf_engine.utils.logger import get_logger

if __name__ == "__main__":
    logger = get_logger()
    logger.info("=========================================")
    logger.info("  SPATIO-TEMPORAL PHYSICS-AI ENGINE V1.0 ")
    logger.info("=========================================")

    pipeline = EnginePipeline()
    pipeline.run_training()

    logger.info("Engine Shutdown Successfully.")

Writing run_engine.py


In [9]:
# @title 1. Initialize System & Directories
import os
import shutil

# Clean slate approach
if os.path.exists("wpf_engine"):
    shutil.rmtree("wpf_engine")

structure = [
    "wpf_engine",
    "wpf_engine/config",
    "wpf_engine/core",  # For evaluation and logic
    "wpf_engine/data",  # For ingestion and generators
    "wpf_engine/models",
    "wpf_engine/pipelines",
    "wpf_engine/utils",
    "saved_models",
    "saved_models/plots",
    "saved_models/logs"
]

for folder in structure:
    os.makedirs(folder, exist_ok=True)
    if "saved_models" not in folder:
        with open(os.path.join(folder, "__init__.py"), "w") as f:
            pass

print("✅ Enterprise Architecture deployed successfully.")

✅ Enterprise Architecture deployed successfully.


In [10]:
# @title 2. Configuration (%%writefile wpf_engine/config/settings.py)
%%writefile wpf_engine/config/settings.py
import os

class Config:
    # -- Data Paths --
    DATA_PATH = "wtbdata_245days.csv"
    ARTIFACTS_DIR = "saved_models"
    PLOT_DIR = os.path.join(ARTIFACTS_DIR, "plots")

    # -- Physics & Time --
    LOOKBACK_STEPS = 144   # 24 hours (144 * 10min)
    FORECAST_STEPS = 288   # 48 hours (288 * 10min)

    # -- Training Hyperparameters --
    BATCH_SIZE = 256       # Large batch for stable gradients
    EPOCHS = 20            # Reduced epochs because dataset is massive (4.7M rows)
    LEARNING_RATE = 1e-3
    TRAIN_TEST_SPLIT = 0.9 # First 90% days for train, last 10% for test

    # -- Feature Definitions --
    TARGET_COL = "Patv"
    # Core physics features calculated by the engine
    PHYSICS_FEATURES = [
        "Wspd", "Wdir", "Etmp", "Itmp", "Ndir", "Pab1", "Pab2", "Pab3", "Prtv",
        "Patv", "U", "V", "Energy_Flux", "Momentum"
    ]

    @classmethod
    def get_model_path(cls):
        return os.path.join(cls.ARTIFACTS_DIR, "deep_physics_transformer.h5")

    @classmethod
    def get_scaler_path(cls):
        return os.path.join(cls.ARTIFACTS_DIR, "global_scaler.pkl")

Writing wpf_engine/config/settings.py


In [11]:
# @title 3. Data Processing (%%writefile wpf_engine/data/processor.py)
%%writefile wpf_engine/data/processor.py
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from wpf_engine.config.settings import Config

class PhysicsEngine:
    @staticmethod
    def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
        print("⚡ Igniting Physics Engine...")

        # 1. Clean Data (KDD Rules)
        df['Patv'] = df['Patv'].clip(lower=0) # Negative power is impossible

        # 2. Vector Decomposition (Solves the 0-360 degree discontinuity)
        wdir_rad = np.deg2rad(df['Wdir'])
        df['U'] = df['Wspd'] * np.cos(wdir_rad)
        df['V'] = df['Wspd'] * np.sin(wdir_rad)

        # 3. Energy Flux (The cubic relationship: Power ~ V^3)
        df['Energy_Flux'] = np.power(df['Wspd'], 3)

        # 4. Momentum (Derivative of wind speed - acceleration)
        # Using specific group-by to avoid bleeding data between turbines
        df['Momentum'] = df.groupby('TurbID')['Wspd'].diff().fillna(0)

        # 5. Handle Missing values securely (Forward fill then Back fill)
        df = df.ffill().bfill()

        return df

class DataManager:
    def __init__(self):
        self.scaler = MinMaxScaler()

    def prepare_data(self, df):
        # Fit scaler on everything to ensure global min/max coverage
        print("⚖️ Scaling 4.7M+ rows (MinMax)...")
        df[Config.PHYSICS_FEATURES] = self.scaler.fit_transform(df[Config.PHYSICS_FEATURES])
        joblib.dump(self.scaler, Config.get_scaler_path())
        return df, self.scaler

class TimeSeriesGenerator(tf.keras.utils.Sequence):
    """
    Zero-Memory-Footprint Generator.
    Instead of creating a 100GB array, we generate batches on the fly.
    """
    def __init__(self, df, indices, batch_size):
        self.df = df
        self.indices = indices
        self.batch_size = batch_size
        self.lookback = Config.LOOKBACK_STEPS
        self.forecast = Config.FORECAST_STEPS
        self.feature_cols = Config.PHYSICS_FEATURES
        self.target_col_idx = df.columns.get_loc(Config.TARGET_COL)
        self.feature_indices = [df.columns.get_loc(c) for c in self.feature_cols]

    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        # Get batch of start indices
        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]

        X_hist, X_anchor, Y_future = [], [], []

        data_array = self.df.values # Access numpy backing once for speed

        for i in batch_indices:
            # 1. Historical Context (0 to t)
            # Shape: (144, Features)
            x_seq = data_array[i : i + self.lookback, self.feature_indices]

            # 2. Power Anchor (at time t)
            # The value the model adds the delta to.
            # Shape: (1,)
            anchor = data_array[i + self.lookback - 1, self.target_col_idx]

            # 3. Future Target (t+1 to t+288)
            # Shape: (288,)
            y_seq = data_array[i + self.lookback : i + self.lookback + self.forecast, self.target_col_idx]

            X_hist.append(x_seq)
            X_anchor.append(anchor)
            Y_future.append(y_seq)

        return [np.array(X_hist), np.array(X_anchor)], np.array(Y_future)

Writing wpf_engine/data/processor.py


In [12]:
# @title 4. Model Architecture (%%writefile wpf_engine/models/architecture.py)
%%writefile wpf_engine/models/architecture.py
import tensorflow as tf
from tensorflow.keras import layers, Model
from wpf_engine.config.settings import Config

def build_differential_model():
    # --- Input 1: The Timeline (History) ---
    input_seq = layers.Input(shape=(Config.LOOKBACK_STEPS, len(Config.PHYSICS_FEATURES)), name="history_in")

    # --- Input 2: The Anchor (Current Power) ---
    input_anchor = layers.Input(shape=(1,), name="anchor_in")

    # 1. Feature Extraction (CNN for local patterns/gusts)
    x = layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(input_seq)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)

    # 2. Temporal Dynamics (Bi-LSTM for long-term memory)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)

    # 3. Differential Prediction Head
    # We predict the *change* relative to the anchor, not absolute values.
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    # Output: Delta for 288 steps
    delta_pred = layers.Dense(Config.FORECAST_STEPS, name="delta_out")(x)

    # 4. Physics Reconstruction (Anchor + Delta)
    # Broadcast anchor to (Batch, 288)
    anchor_broadcast = layers.RepeatVector(Config.FORECAST_STEPS)(input_anchor)
    anchor_broadcast = layers.Reshape((Config.FORECAST_STEPS,))(anchor_broadcast)

    # Final = Anchor + Delta
    final_pred = layers.Add(name="reconstruction")([anchor_broadcast, delta_pred])

    # Physics Constraint: Power >= 0
    final_pred = layers.ReLU()(final_pred)

    model = Model(inputs=[input_seq, input_anchor], outputs=final_pred)

    optimizer = tf.keras.optimizers.Adam(learning_rate=Config.LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

Writing wpf_engine/models/architecture.py


In [13]:
# @title 5. Evaluation & Viz (%%writefile wpf_engine/core/evaluator.py)
%%writefile wpf_engine/core/evaluator.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from wpf_engine.config.settings import Config
import os

class Evaluator:
    def __init__(self, model, scaler):
        self.model = model
        self.scaler = scaler
        # We need the min/max of the Target column to un-scale predictions for real metrics
        self.target_min = scaler.min_[Config.PHYSICS_FEATURES.index(Config.TARGET_COL)]
        self.target_scale = scaler.scale_[Config.PHYSICS_FEATURES.index(Config.TARGET_COL)]

    def unscale(self, data):
        """Converts normalized 0-1 data back to MW/kW."""
        return (data - self.target_min) / self.target_scale

    def evaluate_and_plot(self, test_gen):
        print("📊 Running Comprehensive Evaluation...")

        # 1. Generate Predictions
        # We take a subset of the test generator for visualization (e.g., first 50 batches)
        # to avoid OOM during plotting, but calculate metrics on more.

        all_trues = []
        all_preds = []

        # Iterate over validation set
        steps_to_eval = min(len(test_gen), 100) # Evaluate on ~25k samples
        print(f"   Processing {steps_to_eval} batches...")

        for i in range(steps_to_eval):
            x, y = test_gen[i]
            preds = self.model.predict(x, verbose=0)
            all_trues.append(y)
            all_preds.append(preds)

        # Flatten
        y_true = np.concatenate(all_trues, axis=0)
        y_pred = np.concatenate(all_preds, axis=0)

        # Unscale
        # Since we used MinMaxScaler, X_std = (X - min) / (max - min)
        # X = X_std * (max - min) + min
        # Sklearn: data = (val - min) / scale  ->  val = data * scale + min -> Wait, scale is (max-min) or 1/(max-min)?
        # Sklearn `scale_` attribute is (max-min). Actually: X_scaled = X * scale_ + min_
        # Correction: inverse_transform is safer.

        # Manual inverse for speed on just target
        y_true_real = (y_true - self.scaler.min_[9]) / self.scaler.scale_[9]
        y_pred_real = (y_pred - self.scaler.min_[9]) / self.scaler.scale_[9]

        # --- Metrics ---
        mse = mean_squared_error(y_true_real, y_pred_real)
        mae = mean_absolute_error(y_true_real, y_pred_real)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true_real.flatten(), y_pred_real.flatten())

        print(f"\n🏆 FINAL RESULTS:")
        print(f"   R² Score: {r2:.4f}")
        print(f"   RMSE:     {rmse:.4f} kW")
        print(f"   MAE:      {mae:.4f} kW")
        print(f"   MSE:      {mse:.4f}")

        self.generate_plots(y_true_real, y_pred_real)

        return {"r2": r2, "rmse": rmse, "mae": mae}

    def generate_plots(self, y_true, y_pred):
        plot_dir = Config.PLOT_DIR

        # Plot 1: Prediction vs Truth (Sample)
        plt.figure(figsize=(15, 6))
        # Plot random sample
        idx = np.random.randint(0, len(y_true))
        plt.plot(y_true[idx], label='Ground Truth (Real Physics)', color='black', linewidth=2)
        plt.plot(y_pred[idx], label='AI Prediction', color='dodgerblue', linestyle='--')
        plt.title(f"48-Hour Forecast Horizon (Sample #{idx})")
        plt.xlabel("Time Steps (10-min intervals)")
        plt.ylabel("Active Power (kW)")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{plot_dir}/1_forecast_sample.png")
        plt.close()

        # Plot 2: Error Distribution
        errors = (y_true - y_pred).flatten()
        plt.figure(figsize=(10, 6))
        sns.histplot(errors, bins=100, kde=True, color='crimson')
        plt.title("Error Distribution (Residuals)")
        plt.xlabel("Prediction Error (kW)")
        plt.savefig(f"{plot_dir}/2_error_distribution.png")
        plt.close()

        # Plot 3: Performance Per Horizon Step (Does it get worse over time?)
        # Calculate RMSE per time step (0 to 288)
        rmse_per_step = np.sqrt(np.mean((y_true - y_pred)**2, axis=0))
        plt.figure(figsize=(12, 6))
        plt.plot(rmse_per_step, color='purple')
        plt.title("RMSE Degradation over 48-Hour Horizon")
        plt.xlabel("Forecast Step (10m)")
        plt.ylabel("RMSE (kW)")
        plt.grid(True)
        plt.savefig(f"{plot_dir}/3_rmse_per_step.png")
        plt.close()

        # Plot 4: Scatter Density
        plt.figure(figsize=(8, 8))
        # Sample points to keep plot light
        flat_true = y_true.flatten()
        flat_pred = y_pred.flatten()
        indices = np.random.choice(len(flat_true), size=10000, replace=False)
        plt.scatter(flat_true[indices], flat_pred[indices], alpha=0.1, s=1, c='green')
        plt.plot([0, flat_true.max()], [0, flat_true.max()], 'r--')
        plt.xlabel("Actual Power")
        plt.ylabel("Predicted Power")
        plt.title("Prediction Alignment (R² visual)")
        plt.savefig(f"{plot_dir}/4_scatter_alignment.png")
        plt.close()

        print(f"🎨 Plots saved to {plot_dir}/")

Writing wpf_engine/core/evaluator.py


In [14]:
# @title 6. Training Pipeline (%%writefile wpf_engine/pipelines/run.py)
%%writefile wpf_engine/pipelines/run.py
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from wpf_engine.config.settings import Config
from wpf_engine.data.processor import PhysicsEngine, DataManager, TimeSeriesGenerator
from wpf_engine.models.architecture import build_differential_model
from wpf_engine.core.evaluator import Evaluator

def main():
    print("🚀 Initializing Full Scale WPF Engine...")

    # 1. Load Data
    try:
        df = pd.read_csv(Config.DATA_PATH)
        print(f"📂 Dataset loaded: {df.shape}")
    except FileNotFoundError:
        print("❌ Dataset not found. Please upload 'wtbdata_245days.csv'")
        return

    # 2. Physics & Scaling
    df = PhysicsEngine.engineer_features(df)
    manager = DataManager()
    df, scaler = manager.prepare_data(df)

    # 3. Create Valid Indices for Generator (Crucial for Speed)
    # We filter indices where a full sequence (Lookback + Forecast) stays within one Turbine
    print("⚙️ Computing valid sequence indices...")
    # Group by TurbID and get valid start indices
    groups = df.groupby('TurbID')
    valid_indices = []

    total_len = Config.LOOKBACK_STEPS + Config.FORECAST_STEPS

    # Global index tracking
    for turb_id, group in groups:
        # Get global indices of this group
        g_indices = group.index.values
        # Determine how many valid sequences fit
        n_samples = len(g_indices) - total_len
        if n_samples > 0:
            # We add the start indices.
            # TimeSeriesGenerator will calculate i+lookback based on these.
            valid_starts = g_indices[:n_samples]
            valid_indices.extend(valid_starts)

    valid_indices = np.array(valid_indices)
    np.random.shuffle(valid_indices)

    print(f"✅ Found {len(valid_indices)} valid sequences.")

    # 4. Train/Test Split
    split_idx = int(len(valid_indices) * Config.TRAIN_TEST_SPLIT)
    train_idx = valid_indices[:split_idx]
    test_idx = valid_indices[split_idx:]

    train_gen = TimeSeriesGenerator(df, train_idx, Config.BATCH_SIZE)
    test_gen = TimeSeriesGenerator(df, test_idx, Config.BATCH_SIZE)

    # 5. Build & Train
    model = build_differential_model()

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ModelCheckpoint(Config.get_model_path(), save_best_only=True, monitor='val_loss')
    ]

    print("🔥 Starting GPU Training...")
    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=Config.EPOCHS,
        callbacks=callbacks
    )

    # 6. Evaluate
    evaluator = Evaluator(model, scaler)
    metrics = evaluator.evaluate_and_plot(test_gen)

    print("\n✅ Engine Run Complete.")

if __name__ == "__main__":
    main()

Writing wpf_engine/pipelines/run.py


In [15]:
# @title 1. Fix Pathing in Pipeline Script (%%writefile wpf_engine/pipelines/run.py)
%%writefile wpf_engine/pipelines/run.py
import sys
import os

# --- PATH PATCH: Auto-detect Root Directory ---
# This allows the script to be run directly from anywhere
current_dir = os.path.dirname(os.path.abspath(__file__)) # .../wpf_engine/pipelines
root_dir = os.path.abspath(os.path.join(current_dir, '../..')) # .../
if root_dir not in sys.path:
    sys.path.append(root_dir)
# ----------------------------------------------

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from wpf_engine.config.settings import Config
from wpf_engine.data.processor import PhysicsEngine, DataManager, TimeSeriesGenerator
from wpf_engine.models.architecture import build_differential_model
from wpf_engine.core.evaluator import Evaluator

def main():
    print(f"🚀 Initializing Full Scale WPF Engine (Root: {root_dir})...")

    # 1. Load Data
    try:
        df = pd.read_csv(Config.DATA_PATH)
        print(f"📂 Dataset loaded: {df.shape}")
    except FileNotFoundError:
        print(f"❌ Dataset not found at {Config.DATA_PATH}. Please upload 'wtbdata_245days.csv' to Colab Files.")
        return

    # 2. Physics & Scaling
    df = PhysicsEngine.engineer_features(df)
    manager = DataManager()
    df, scaler = manager.prepare_data(df)

    # 3. Create Valid Indices for Generator (Crucial for Speed)
    # We filter indices where a full sequence (Lookback + Forecast) stays within one Turbine
    print("⚙️ Computing valid sequence indices...")
    # Group by TurbID and get valid start indices
    groups = df.groupby('TurbID')
    valid_indices = []

    total_len = Config.LOOKBACK_STEPS + Config.FORECAST_STEPS

    # Global index tracking
    for turb_id, group in groups:
        # Get global indices of this group
        g_indices = group.index.values
        # Determine how many valid sequences fit
        n_samples = len(g_indices) - total_len
        if n_samples > 0:
            # We add the start indices.
            # TimeSeriesGenerator will calculate i+lookback based on these.
            valid_starts = g_indices[:n_samples]
            valid_indices.extend(valid_starts)

    valid_indices = np.array(valid_indices)
    np.random.shuffle(valid_indices)

    print(f"✅ Found {len(valid_indices)} valid sequences.")

    # 4. Train/Test Split
    split_idx = int(len(valid_indices) * Config.TRAIN_TEST_SPLIT)
    train_idx = valid_indices[:split_idx]
    test_idx = valid_indices[split_idx:]

    print(f"   Training Batches: {len(train_idx) // Config.BATCH_SIZE}")
    print(f"   Testing Batches:  {len(test_idx) // Config.BATCH_SIZE}")

    train_gen = TimeSeriesGenerator(df, train_idx, Config.BATCH_SIZE)
    test_gen = TimeSeriesGenerator(df, test_idx, Config.BATCH_SIZE)

    # 5. Build & Train
    model = build_differential_model()

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ModelCheckpoint(Config.get_model_path(), save_best_only=True, monitor='val_loss')
    ]

    print("🔥 Starting GPU Training...")
    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=Config.EPOCHS,
        callbacks=callbacks
    )

    # 6. Evaluate
    evaluator = Evaluator(model, scaler)
    metrics = evaluator.evaluate_and_plot(test_gen)

    print("\n✅ Engine Run Complete.")

if __name__ == "__main__":
    main()

Overwriting wpf_engine/pipelines/run.py


In [16]:
# @title 1. Create Root Entry Point (%%writefile main.py)
%%writefile main.py
import sys
import os

# FORCE: Add the current directory to Python's path so it finds 'wpf_engine'
sys.path.append(os.getcwd())

from wpf_engine.pipelines.run import main
from wpf_engine.config.settings import Config

if __name__ == "__main__":
    print(f"🔧 System Root: {os.getcwd()}")
    print(f"🔧 Target Dataset: {Config.DATA_PATH}")

    # Run the full pipeline
    main()

Writing main.py


In [17]:
# @title 1. Apply Float32 Fix (%%writefile wpf_engine/data/processor.py)
%%writefile wpf_engine/data/processor.py
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from wpf_engine.config.settings import Config

class PhysicsEngine:
    @staticmethod
    def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
        print("⚡ Igniting Physics Engine...")

        # 1. Clean Data
        df['Patv'] = df['Patv'].clip(lower=0)

        # 2. Vector Decomposition
        wdir_rad = np.deg2rad(df['Wdir'])
        df['U'] = df['Wspd'] * np.cos(wdir_rad)
        df['V'] = df['Wspd'] * np.sin(wdir_rad)

        # 3. Energy Flux
        df['Energy_Flux'] = np.power(df['Wspd'], 3)

        # 4. Momentum
        df['Momentum'] = df.groupby('TurbID')['Wspd'].diff().fillna(0)

        # 5. Fill NaNs
        df = df.ffill().bfill()

        return df

class DataManager:
    def __init__(self):
        self.scaler = MinMaxScaler()

    def prepare_data(self, df):
        print("⚖️ Scaling 4.7M+ rows (MinMax)...")
        # Scale only the physics columns
        df[Config.PHYSICS_FEATURES] = self.scaler.fit_transform(df[Config.PHYSICS_FEATURES])
        joblib.dump(self.scaler, Config.get_scaler_path())
        return df, self.scaler

class TimeSeriesGenerator(tf.keras.utils.Sequence):
    """
    Fixed Generator: Explicitly extracts float32 data to prevent Object dtype errors.
    """
    def __init__(self, df, indices, batch_size, **kwargs):
        super().__init__(**kwargs) # Fixes the UserWarning
        self.indices = indices
        self.batch_size = batch_size
        self.lookback = Config.LOOKBACK_STEPS
        self.forecast = Config.FORECAST_STEPS

        # --- CRITICAL FIX ---
        # 1. We only extract the specific columns defined in settings
        # 2. We explicitly cast to float32 to satisfy the GPU
        self.data_matrix = df[Config.PHYSICS_FEATURES].values.astype(np.float32)

        # Map column names to indices in this new compact matrix
        self.feature_indices = [Config.PHYSICS_FEATURES.index(c) for c in Config.PHYSICS_FEATURES]
        self.target_idx = Config.PHYSICS_FEATURES.index(Config.TARGET_COL)
        # --------------------

    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        # Batch of start indices
        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]

        # Pre-allocate arrays for speed (Vectorization)
        X_hist = np.empty((self.batch_size, self.lookback, len(self.feature_indices)), dtype=np.float32)
        X_anchor = np.empty((self.batch_size, 1), dtype=np.float32)
        Y_future = np.empty((self.batch_size, self.forecast), dtype=np.float32)

        for k, i in enumerate(batch_indices):
            # 1. Historical Context
            X_hist[k] = self.data_matrix[i : i + self.lookback]

            # 2. Power Anchor
            X_anchor[k] = self.data_matrix[i + self.lookback - 1, self.target_idx]

            # 3. Future Target
            Y_future[k] = self.data_matrix[i + self.lookback : i + self.lookback + self.forecast, self.target_idx]

        return [X_hist, X_anchor], Y_future

Overwriting wpf_engine/data/processor.py


In [18]:
# @title 1. Apply Dictionary Input Fix (%%writefile wpf_engine/data/processor.py)
%%writefile wpf_engine/data/processor.py
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from wpf_engine.config.settings import Config

class PhysicsEngine:
    @staticmethod
    def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
        print("⚡ Igniting Physics Engine...")
        df['Patv'] = df['Patv'].clip(lower=0)
        wdir_rad = np.deg2rad(df['Wdir'])
        df['U'] = df['Wspd'] * np.cos(wdir_rad)
        df['V'] = df['Wspd'] * np.sin(wdir_rad)
        df['Energy_Flux'] = np.power(df['Wspd'], 3)
        df['Momentum'] = df.groupby('TurbID')['Wspd'].diff().fillna(0)
        df = df.ffill().bfill()
        return df

class DataManager:
    def __init__(self):
        self.scaler = MinMaxScaler()

    def prepare_data(self, df):
        print("⚖️ Scaling 4.7M+ rows (MinMax)...")
        df[Config.PHYSICS_FEATURES] = self.scaler.fit_transform(df[Config.PHYSICS_FEATURES])
        joblib.dump(self.scaler, Config.get_scaler_path())
        return df, self.scaler

class TimeSeriesGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, indices, batch_size, **kwargs):
        super().__init__(**kwargs)
        self.indices = indices
        self.batch_size = batch_size
        self.lookback = Config.LOOKBACK_STEPS
        self.forecast = Config.FORECAST_STEPS

        # 1. Select only physics columns
        # 2. Force float32 to prevent Object dtype errors
        self.data_matrix = df[Config.PHYSICS_FEATURES].values.astype(np.float32)

        self.feature_indices = [Config.PHYSICS_FEATURES.index(c) for c in Config.PHYSICS_FEATURES]
        self.target_idx = Config.PHYSICS_FEATURES.index(Config.TARGET_COL)

    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]

        X_hist = np.empty((self.batch_size, self.lookback, len(self.feature_indices)), dtype=np.float32)
        X_anchor = np.empty((self.batch_size, 1), dtype=np.float32)
        Y_future = np.empty((self.batch_size, self.forecast), dtype=np.float32)

        for k, i in enumerate(batch_indices):
            X_hist[k] = self.data_matrix[i : i + self.lookback]
            X_anchor[k] = self.data_matrix[i + self.lookback - 1, self.target_idx]
            Y_future[k] = self.data_matrix[i + self.lookback : i + self.lookback + self.forecast, self.target_idx]

        # FIX: Return Dictionary mapping input layer names -> Data
        # This matches names defined in wpf_engine/models/architecture.py
        inputs = {
            "history_in": X_hist,
            "anchor_in": X_anchor
        }

        return inputs, Y_future

Overwriting wpf_engine/data/processor.py


In [19]:
# @title 1. Optimize Settings (%%writefile wpf_engine/config/settings.py)
%%writefile wpf_engine/config/settings.py
import os

class Config:
    # -- Data Paths --
    DATA_PATH = "wtbdata_245days.csv"
    ARTIFACTS_DIR = "saved_models"
    PLOT_DIR = os.path.join(ARTIFACTS_DIR, "plots")

    # -- Physics & Time --
    LOOKBACK_STEPS = 144
    FORECAST_STEPS = 288

    # -- OPTIMIZATION SETTINGS --
    BATCH_SIZE = 2048      # INCREASED 8x (From 256)
    EPOCHS = 30
    LEARNING_RATE = 1e-3
    TRAIN_TEST_SPLIT = 0.9

    # -- Feature Definitions --
    TARGET_COL = "Patv"
    PHYSICS_FEATURES = [
        "Wspd", "Wdir", "Etmp", "Itmp", "Ndir", "Pab1", "Pab2", "Pab3", "Prtv",
        "Patv", "U", "V", "Energy_Flux", "Momentum"
    ]

    @classmethod
    def get_model_path(cls):
        return os.path.join(cls.ARTIFACTS_DIR, "deep_physics_transformer.h5")

    @classmethod
    def get_scaler_path(cls):
        return os.path.join(cls.ARTIFACTS_DIR, "global_scaler.pkl")

Overwriting wpf_engine/config/settings.py


In [20]:
# @title 2. Apply Vectorized Generator (%%writefile wpf_engine/data/processor.py)
%%writefile wpf_engine/data/processor.py
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from wpf_engine.config.settings import Config

class PhysicsEngine:
    @staticmethod
    def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
        print("⚡ Igniting Physics Engine...")
        df['Patv'] = df['Patv'].clip(lower=0)
        wdir_rad = np.deg2rad(df['Wdir'])
        df['U'] = df['Wspd'] * np.cos(wdir_rad)
        df['V'] = df['Wspd'] * np.sin(wdir_rad)
        df['Energy_Flux'] = np.power(df['Wspd'], 3)
        df['Momentum'] = df.groupby('TurbID')['Wspd'].diff().fillna(0)
        df = df.ffill().bfill()
        return df

class DataManager:
    def __init__(self):
        self.scaler = MinMaxScaler()

    def prepare_data(self, df):
        print("⚖️ Scaling 4.7M+ rows (MinMax)...")
        df[Config.PHYSICS_FEATURES] = self.scaler.fit_transform(df[Config.PHYSICS_FEATURES])
        joblib.dump(self.scaler, Config.get_scaler_path())
        return df, self.scaler

class TimeSeriesGenerator(tf.keras.utils.Sequence):
    """
    Highly Optimized Vectorized Generator.
    Removes Python loops to maximize GPU throughput.
    """
    def __init__(self, df, indices, batch_size, **kwargs):
        super().__init__(**kwargs)
        self.indices = indices
        self.batch_size = batch_size
        self.lookback = Config.LOOKBACK_STEPS
        self.forecast = Config.FORECAST_STEPS

        # Matrix is loaded into RAM once
        self.data_matrix = df[Config.PHYSICS_FEATURES].values.astype(np.float32)
        self.target_idx = Config.PHYSICS_FEATURES.index(Config.TARGET_COL)

        # Pre-calculate offset arrays for broadcasting
        # This creates the "sliding window" indices in C-speed
        self.lookback_offsets = np.arange(self.lookback)
        self.forecast_offsets = np.arange(self.forecast) + self.lookback

    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        # 1. Get the start indices for this batch
        starts = self.indices[index*self.batch_size : (index+1)*self.batch_size]

        # 2. Vectorized Slicing (Magic happens here)
        # Create a grid of indices: (Batch_Size, Lookback_Steps)
        # This replaces the 'for' loop entirely
        history_idx = starts[:, None] + self.lookback_offsets
        forecast_idx = starts[:, None] + self.forecast_offsets
        anchor_idx = starts + self.lookback - 1

        # 3. Bulk Fetch from RAM
        X_hist = self.data_matrix[history_idx] # Shape: (Batch, 144, Features)
        X_anchor = self.data_matrix[anchor_idx, self.target_idx] # Shape: (Batch,)
        Y_future = self.data_matrix[forecast_idx, self.target_idx] # Shape: (Batch, 288)

        # 4. Reshape Anchor to (Batch, 1) to match Model Input
        X_anchor = X_anchor[:, None]

        inputs = {
            "history_in": X_hist,
            "anchor_in": X_anchor
        }

        return inputs, Y_future

Overwriting wpf_engine/data/processor.py


In [21]:
# @title 3. Update Pipeline (%%writefile wpf_engine/pipelines/run.py)
%%writefile wpf_engine/pipelines/run.py
import sys
import os

# --- PATH PATCH ---
current_dir = os.path.dirname(os.path.abspath(__file__))
root_dir = os.path.abspath(os.path.join(current_dir, '../..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
# ------------------

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from wpf_engine.config.settings import Config
from wpf_engine.data.processor import PhysicsEngine, DataManager, TimeSeriesGenerator
from wpf_engine.models.architecture import build_differential_model
from wpf_engine.core.evaluator import Evaluator

def main():
    print(f"🚀 Initializing HIGH-PERFORMANCE Engine (Batch Size: {Config.BATCH_SIZE})...")

    # Load & Process
    try:
        df = pd.read_csv(Config.DATA_PATH)
    except FileNotFoundError:
        print("❌ Dataset not found.")
        return

    df = PhysicsEngine.engineer_features(df)
    manager = DataManager()
    df, scaler = manager.prepare_data(df)

    # Indices Logic
    print("⚙️ Computing valid sequence indices...")
    groups = df.groupby('TurbID')
    valid_indices = []
    total_len = Config.LOOKBACK_STEPS + Config.FORECAST_STEPS

    for turb_id, group in groups:
        g_indices = group.index.values
        n_samples = len(g_indices) - total_len
        if n_samples > 0:
            valid_indices.extend(g_indices[:n_samples])

    valid_indices = np.array(valid_indices)
    np.random.shuffle(valid_indices)
    print(f"✅ Found {len(valid_indices)} valid sequences.")

    # Split
    split_idx = int(len(valid_indices) * Config.TRAIN_TEST_SPLIT)
    train_idx = valid_indices[:split_idx]
    test_idx = valid_indices[split_idx:]

    # Generators
    train_gen = TimeSeriesGenerator(df, train_idx, Config.BATCH_SIZE)
    test_gen = TimeSeriesGenerator(df, test_idx, Config.BATCH_SIZE)

    # Model
    model = build_differential_model()

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
        ModelCheckpoint(Config.get_model_path(), save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)
    ]

    print(f"🔥 Starting GPU Training (Batch Size {Config.BATCH_SIZE})...")

    # --- PERFORMANCE FIX: Multiprocessing ---
    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=Config.EPOCHS,
        callbacks=callbacks,
        workers=4,                # Use 4 CPU cores to fetch data
        use_multiprocessing=True, # Parallelize the Generator
        max_queue_size=20         # Buffer more batches in RAM
    )

    # Evaluate
    evaluator = Evaluator(model, scaler)
    metrics = evaluator.evaluate_and_plot(test_gen)
    print("\n✅ Engine Run Complete.")

if __name__ == "__main__":
    main()

Overwriting wpf_engine/pipelines/run.py


In [22]:
# @title 1. Fix Keras 3 Compatibility (%%writefile wpf_engine/pipelines/run.py)
%%writefile wpf_engine/pipelines/run.py
import sys
import os

# --- PATH PATCH ---
current_dir = os.path.dirname(os.path.abspath(__file__))
root_dir = os.path.abspath(os.path.join(current_dir, '../..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
# ------------------

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from wpf_engine.config.settings import Config
from wpf_engine.data.processor import PhysicsEngine, DataManager, TimeSeriesGenerator
from wpf_engine.models.architecture import build_differential_model
from wpf_engine.core.evaluator import Evaluator

def main():
    print(f"🚀 Initializing HIGH-PERFORMANCE Engine (Batch Size: {Config.BATCH_SIZE})...")

    # Load & Process
    try:
        df = pd.read_csv(Config.DATA_PATH)
    except FileNotFoundError:
        print("❌ Dataset not found.")
        return

    df = PhysicsEngine.engineer_features(df)
    manager = DataManager()
    df, scaler = manager.prepare_data(df)

    # Indices Logic
    print("⚙️ Computing valid sequence indices...")
    groups = df.groupby('TurbID')
    valid_indices = []
    total_len = Config.LOOKBACK_STEPS + Config.FORECAST_STEPS

    for turb_id, group in groups:
        g_indices = group.index.values
        n_samples = len(g_indices) - total_len
        if n_samples > 0:
            valid_indices.extend(g_indices[:n_samples])

    valid_indices = np.array(valid_indices)
    np.random.shuffle(valid_indices)
    print(f"✅ Found {len(valid_indices)} valid sequences.")

    # Split
    split_idx = int(len(valid_indices) * Config.TRAIN_TEST_SPLIT)
    train_idx = valid_indices[:split_idx]
    test_idx = valid_indices[split_idx:]

    # Generators (Vectorized = Fast)
    train_gen = TimeSeriesGenerator(df, train_idx, Config.BATCH_SIZE)
    test_gen = TimeSeriesGenerator(df, test_idx, Config.BATCH_SIZE)

    # Model
    model = build_differential_model()

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
        ModelCheckpoint(Config.get_model_path(), save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)
    ]

    print(f"🔥 Starting GPU Training (Batch Size {Config.BATCH_SIZE})...")

    # --- KERAS 3 FIX: Removed 'workers' and 'use_multiprocessing' ---
    # The Vectorized Generator is fast enough to run on the main thread
    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=Config.EPOCHS,
        callbacks=callbacks
    )

    # Evaluate
    evaluator = Evaluator(model, scaler)
    metrics = evaluator.evaluate_and_plot(test_gen)
    print("\n✅ Engine Run Complete.")

if __name__ == "__main__":
    main()

Overwriting wpf_engine/pipelines/run.py


In [23]:
# @title 1. Maximize Settings (%%writefile wpf_engine/config/settings.py)
%%writefile wpf_engine/config/settings.py
import os

class Config:
    # -- Data Paths --
    DATA_PATH = "wtbdata_245days.csv"
    ARTIFACTS_DIR = "saved_models"
    PLOT_DIR = os.path.join(ARTIFACTS_DIR, "plots")

    # -- Physics & Time --
    LOOKBACK_STEPS = 144
    FORECAST_STEPS = 288

    # -- ULTRA SETTINGS --
    BATCH_SIZE = 4096      # Pushing to 4096 to fill VRAM
    EPOCHS = 30
    LEARNING_RATE = 5e-4   # Slightly lower LR for larger batches
    TRAIN_TEST_SPLIT = 0.9

    # -- Feature Definitions --
    TARGET_COL = "Patv"
    PHYSICS_FEATURES = [
        "Wspd", "Wdir", "Etmp", "Itmp", "Ndir", "Pab1", "Pab2", "Pab3", "Prtv",
        "Patv", "U", "V", "Energy_Flux", "Momentum"
    ]

    @classmethod
    def get_model_path(cls):
        return os.path.join(cls.ARTIFACTS_DIR, "deep_physics_transformer.h5")

    @classmethod
    def get_scaler_path(cls):
        return os.path.join(cls.ARTIFACTS_DIR, "global_scaler.pkl")

Overwriting wpf_engine/config/settings.py
